### Classification approach for exercise

In [3]:
import os,re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import classification_report,confusion_matrix,log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

pd.set_option('display.max_columns',200)
pd.set_option("display.max_rows",500)

In [5]:
dfTrain = pd.read_csv("exercise_40_train.csv")
dfTest = pd.read_csv("exercise_40_test.csv")
dfTest.head(15)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,4.747627,20.509439,Wednesday,2.299105,-1.815777,-0.752166,0.0098%,-3.240309,0.587948,-0.260721,101.113628,-0.812035,3.251085,-0.004432,-0.917763,102.193597,7.097387,75.256714,$120.216189955777,8.269754,4.794438,56.134458,-0.083641,NaN,5.740955,-3.152540,924.013304,17.697125,1.402273,NaN,no,1.461654,Florida,57.930285,4.727300,66.416594,28.450466,33.499310,5-10 miles,16.776061,99971.844703,0.492812,-0.963616,NaN,NaN,1.266416,53.020326,61.840284,NaN,1825.413159,2.517924,NaN,70.880778,NaN,1.923125,0.239009,NaN,300.629990,0,May,5.461123,5.149473,1.179229,59.346822,progressive,2.219502,17.667792,5.557066,-2.030901,0.636111,-0.000234,62.363381,4.613878,2.415655,3.632041,-0.382482,mercedes,55.938387,1.0,8.325299,-1.274085,-5.663245,2.402660,-0.061040,NaN,64.002500,0.548974,3.420875,11.553023,5.904644,NaN,12.542333,no,3.107683,0.533904,12.438759,7.298306,0,NaN,93.567120
1,1.148654,19.301465,Fri,1.862200,-0.773707,-1.461276,0.0076%,0.443209,0.522113,-1.090886,104.791999,8.805876,1.651993,NaN,-1.396065,99.356609,7.117930,79.061540,$-267.562586413086,5.668347,12.377178,NaN,0.321981,female,2.058123,-0.442525,1107.628399,15.747234,2.027073,NaN,yes,0.608259,North Carolina,55.646392,0.789132,68.881807,32.242558,-1.034680,5-10 miles,11.959804,99910.554923,0.547935,1.001799,1.315020,3.229084,5.873890,49.116516,61.279131,9.360134,1818.390072,2.269700,0.336755,69.832156,2.666487,1.224195,-1.214940,NaN,300.629990,0,July,6.520113,2.101449,-0.871509,NaN,allstate,1.289800,14.402071,7.302161,-1.553879,1.352019,-0.000234,54.104054,9.010198,0.537178,1.489493,-1.106853,ford,44.322947,1.0,6.088895,1.878944,-8.237793,3.636347,3.726452,105.488589,53.387201,-0.751229,0.295234,6.212811,4.876645,-0.848567,7.213829,yes,4.276078,NaN,10.386987,12.527094,1,yes,98.607486
2,4.986860,18.769675,Saturday,1.040845,-1.548690,2.632948,-5e-04%,-1.167885,5.739275,0.222975,102.109546,7.831517,3.055358,2.036434,1.057296,NaN,10.943217,87.567827,$-311.292903116571,3.219583,31.135956,50.048638,1.174485,male,1.609278,-0.303259,1009.911508,12.008190,0.618778,NaN,no,0.680923,NaN,53.178113,0.869321,70.249633,35.207243,48.980294,5-10 miles,14.564732,100729.380783,0.096947,-0.490053,NaN,1.333292,0.750075,48.258898,63.737244,11.564194,1815.680559,1.704048,NaN,67.090400,1.547230,NaN,1.428580,NaN,300.629990,0,January,12.190433,1.793349,-0.114922,48.121885,progressive,-1.755707,14.402071,9.903803,1.720469,0.765756,-0.000234,51.522621,11.700359,-1.867170,0.383319,-1.078648,NaN,48.854080,1.0,8.711055,-0.073306,-8.739095,NaN,2.138771,NaN,49.687134,2.641871,1.718243,NaN,4.455432,1.143388,10.483928,no,2.090868,-1.780474,11.328177,11.628247,0,yes,94.578246
3,3.709183,18.374375,Tuesday,-0.169882,-2.396549,-0.784673,-0.016%,-2.662226,1.548050,0.210141,82.653354,0.436885,1.578106,NaN,-1.287913,102.410965,6.588790,71.825782,$2229.14940030076,7.459929,1.822459,88.144007,0.909556,female,8.864059,0.641209,841.889126,20.904196,0.725017,NaN,no,0.622849,Mississippi,50.311869,0.453211,65.253390,34.432292,52.756665,5-10 miles,18.503815,101476.778846,0.888038,-0.007376,-1.126059,1.129508,-0.455920,44.525657,60.008453,12.852088,2251.680231,2.915405,5.895661,75.219207,NaN,NaN,-0.415800,NaN,300.629990,0,July,6.865209,5.083537,1.685063,46.761738,geico,-0.807993,14.402071,16.576216,0.033036,0.284538,-0.000234,54.625974,13.160347,-0.329204,2.171326,-0.109125,subaru,NaN,1.0,11.742605,-0.253294,-6.641284,4.755348,2.138771,NaN,NaN,1.811825,0.461637,18.198978,3.947223,0.693646,3.862867,no,2.643847,1.662240,10.064961,10.550014,1,NaN,100.346261
4,3.801616,20.205541,Monday,2.092652,-0.732784,-0.703101,0.0186%,0.056422,2.878167,-0.45

### Combine data frames

#### Goal is to encode categorical variables for train & test

In [7]:
dfCombine = pd.concat([dfTrain,dfTest])
dfCombine.iloc[40000:40001,]

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,NaN,4.747627,20.509439,Wednesday,2.299105,-1.815777,-0.752166,0.0098%,-3.240309,0.587948,-0.260721,101.113628,-0.812035,3.251085,-0.004432,-0.917763,102.193597,7.097387,75.256714,$120.216189955777,8.269754,4.794438,56.134458,-0.083641,NaN,5.740955,-3.15254,924.013304,17.697125,1.402273,NaN,no,1.461654,Florida,57.930285,4.7273,66.416594,28.450466,33.49931,5-10 miles,16.776061,99971.844703,0.492812,-0.963616,NaN,NaN,1.266416,53.020326,61.840284,NaN,1825.413159,2.517924,NaN,70.880778,NaN,1.923125,0.239009,NaN,300.62999,0,May,5.461123,5.149473,1.179229,59.346822,progressive,2.219502,17.667792,5.557066,-2.030901,0.636111,-0.000234,62.363381,4.613878,2.415655,3.632041,-0.382482,mercedes,55.938387,1.0,8.325299,-1.274085,-5.663245,2.40266,-0.06104,NaN,64.0025,0.548974,3.420875,11.553023,5.904644,NaN,12.542333,no,3.107683,0.533904,12.438759,7.298306,0,NaN,93.56712


In [8]:
dfCombine.reset_index(drop=True,inplace=True)
dfCombine.tail(1)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
49999,NaN,2.972268,20.485504,Sunday,1.747878,0.197357,1.798676,0.0086%,-0.318751,2.858806,0.08416,99.642148,8.497151,0.878065,-0.338883,0.350812,101.065927,7.900097,76.79105,$-508.049552925335,0.443446,-0.255657,82.889762,-2.112773,female,2.764069,NaN,1028.57299,18.45714,2.473741,-0.578496,yes,0.634841,NaN,50.916587,1.271283,63.837508,29.870539,39.909804,5-10 miles,19.938424,101720.09239,-0.875818,0.765511,NaN,0.996541,-3.489898,49.483851,65.862504,7.303529,1150.95107,1.808276,9.110502,72.228876,-0.61096,2.634673,-1.92103,NaN,300.62999,0,September,6.847741,2.181149,1.297841,50.885163,progressive,0.153248,14.402071,11.865657,-1.29376,0.434589,-0.000234,52.004365,4.865971,NaN,1.524051,0.731896,NaN,45.800511,1.0,7.780886,0.953972,-6.183223,1.565181,3.140183,95.665994,NaN,-0.127836,-1.550846,11.222902,4.651827,1.397517,8.463199,yes,3.942066,1.245289,NaN,11.718582,0,yes,98.307681


### Clean data

#### Removal of null values

In [9]:
dfCombine.isnull().sum()

y       10000
x1          0
x2          0
x3          0
x4          0
x5       3030
x6          0
x7          0
x8          0
x9          0
x10         0
x11      6439
x12         0
x13         0
x14     12292
x15         0
x16     13965
x17         0
x18         0
x19         0
x20         0
x21         0
x22      3000
x23         0
x24      4825
x25         0
x26      3050
x27         0
x28         0
x29         0
x30     40421
x31         0
x32         0
x33      8941
x34         0
x35         0
x36         0
x37         0
x38      2967
x39         0
x40         0
x41     11907
x42     12147
x43         0
x44     42813
x45     10072
x46         0
x47         0
x48         0
x49     16077
x50         0
x51         0
x52     20262
x53         0
x54     15934
x55     22120
x56         0
x57     40541
x58         0
x59         0
x60         0
x61      9059
x62         0
x63      3007
x64      6363
x65         0
x66         0
x67      3045
x68      2984
x69         0
x70         0
x71   

### Drop columns with null values & other un-needed columns

In [10]:
# dfCombine.drop(["x5","x11","x14","x16","x22","x24","x26","x30","x33","x38","x41","x42","x44","x45","x49","x52","x54",\
# "x55","x57","x61","x63","x64","x67","x68","x74","x75","x76","x77","x78","x79","x80","x85","x89","x95","x99",\
# "x83","x86","x88","x91","x92","x94","x96"],axis=1,inplace=True)
# unneeded: x39
dfCombine.drop(["x11","x14","x16","x24","x30","x33","x38","x39","x41","x42","x44","x45","x49","x52","x54",\
"x55","x57","x61","x64","x74","x75","x76","x77","x78","x79","x80","x85","x89","x95","x99",\
"x83","x86","x88","x91","x96"],axis=1,inplace=True)

In [11]:
dfCombine.isnull().sum()

y       10000
x1          0
x2          0
x3          0
x4          0
x5       3030
x6          0
x7          0
x8          0
x9          0
x10         0
x12         0
x13         0
x15         0
x17         0
x18         0
x19         0
x20         0
x21         0
x22      3000
x23         0
x25         0
x26      3050
x27         0
x28         0
x29         0
x31         0
x32         0
x34         0
x35         0
x36         0
x37         0
x40         0
x43         0
x46         0
x47         0
x48         0
x50         0
x51         0
x53         0
x56         0
x58         0
x59         0
x60         0
x62         0
x63      3007
x65         0
x66         0
x67      3045
x68      2984
x69         0
x70         0
x71         0
x72         0
x73         0
x81         0
x82         0
x84         0
x87         0
x90         0
x92      3061
x93         0
x94      2955
x97         0
x98         0
x100        0
dtype: int64

In [13]:
dfCombine[["x5","x22","x26","x63","x67","x68","x92","x94"]].sample(30)

,x5,x22,x26,x63,x67,x68,x92,x94
21763,NaN,19.470680,NaN,-1.274969,14.402071,10.029268,1.852088,0.633346
27925,-1.034614,86.954794,-1.207133,0.175002,14.402071,NaN,8.395535,3.295258
12731,1.353793,155.434208,2.040047,0.399203,14.402071,5.258130,4.369000,3.359061
34984,0.874903,29.957684,-0.177468,-0.892646,NaN,19.819020,3.587513,2.138184
5811,-0.141493,30.563129,0.390292,0.105798,14.402071,21.162576,3.043976,2.662600
29576,1.324089,43.528315,3.219540,0.876129,14.402071,23.717844,5.336864,4.719040
34851,2.016772,59.978897,0.535555,0.569671,14.402071,21.025405,2.090957,2.329912
11780,-0.000970,71.073110,2.575249,-1.148309,17.504057,19.307316,NaN,2.431110
22476,2.028108,56.475893,3.107794,-2.686838,6.849080,16.258699,2.827361,5.551139
25358,1.627839,177.917257,1.991796,-1.131329,14.402071,17.530436,2.551657,NaN


### Interpolate missing values

In [14]:
dfCombine.interpolate(method="linear",inplace=True)
dfCombine.sample(25)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
24802,1.0,3.700801,19.555568,Monday,-1.532088,-0.923755,0.111407,-0.001%,-1.182973,8.051184,1.041465,7.703282,7.457535,1.340633,10.273423,79.730473,$-359.607965977699,0.386486,42.339582,86.067256,0.287248,2.715120,0.329911,1084.704197,13.803835,0.673830,no,0.740073,48.646312,3.192271,69.296274,34.133545,18.847269,2.701746,-0.411013,53.403187,60.512650,517.394720,0.540429,74.772802,1.101605,300.629990,0,January,0.934478,1.495647,esurance,-2.252786,14.402071,15.468054,1.152684,0.538734,-0.000234,36.738089,6.654212,4.275297,-6.476382,2.138771,-0.942958,2.745541,3.750902,no,5.302055,10.767334,1,99.938765
43069,0.0,1.649476,19.615851,Tuesday,-1.115213,2.141787,-1.129252,-0.006%,0.569842,2.571683,1.035530,6.862350,1.606349,-0.107158,5.261893,78.618180,$1874.24369341207,2.228050,14.460524,35.798236,-0.653463,6.985210,2.080092,942.319345,16.597532,1.710020,no,1.018282,53.567341,2.703057,64.583384,27.691142,18.811187,0.845235,0.032445,50.544070,62.941513,2779.280859,3.306840,67.694039,2.661034,300.629990,0,August,3.933322,0.607703,progressive,1.665065,14.402071,7.269848,1.960180,0.750156,-0.000234,43.148572,0.839537,-1.980218,-7.225278,2.138771,0.121360,3.590510,2.853748,no,5.125246,10.758023,0,104.537450
39750,0.0,4.439545,18.472721,Saturday,0.363669,-0.169525,-0.616954,0.014%,-1.528909,3.701774,-0.136310,8.338010,3.874768,0.517432,7.920948,82.882179,$-160.682643814916,6.330114,2.666357,40.105664,-0.950226,6.199639,1.531637,917.920595,18.679126,2.188473,no,1.936681,55.122872,1.559101,65.424637,35.547435,17.604841,-1.620355,-0.304629,43.945576,61.427916,3402.489377,0.775183,71.778209,1.081137,300.629990,0,January,3.861655,0.791627,allstate,-1.005737,12.374366,12.875379,1.316684,0.668292,-0.000234,41.154967,6.464509,0.803317,-8.536627,2.138771,1.400996,4.464492,4.532295,no,5.791376,10.050135,1,103.144357
17480,0.0,4.599109,19.742917,Thur,-0.557934,-0.329235,0.970999,0.0101%,0.271654,6.380765,-0.191324,1.844777,4.559143,0.608429,12.311827,69.838651,$-1592.02868516967,2.865914,26.262056,110.323440,-1.033847,6.449892,0.553887,839.847183,19.538627,0.036430,no,16.629044,57.797117,3.778507,63.868332,32.951233,10.739881,2.404445,3.608758,44.355341,61.159868,3105.255136,0.175709,60.723092,0.291807,300.629990,0,December,2.605840,0.038241,allstate,-1.213188,14.402071,12.037462,1.208013,1.390743,-0.000234,43.024559,9.912208,0.040941,-7.025224,2.138771,0.031223,2.802617,0.540575,no,2.676735,9.349418,1,102.021334
7325,0.0,1.735975,20.905010,Sat,-2.070306,0.627281,0.974494,0.0097%,-0.320903,5.200087,-0.233427,-0.923607,5.638686,0.936419,11.656277,90.415711,$-1043.61820749921,1.310010,53.764337,92.091258,2.644387,4.576018,-0.522856,907.013954,9.516653,0.227246,no,1.580127,63.742716,2.019339,77.241932,26.916440,19.417938,0.886280,-1.831387,57.912968,66.158458,590.272561,1.291249,82.732346,-2.430034,300.629990,0,January,12.589765,1.944357,farmers,-0.476885,14.402071,5.262208,-2.065639,0.065304,-0.000234,45.206988,6.361235,-1.555993,-9.402624,2.138771,-0.995613,1.140568,6.901240,no,2.516500,9.283714,0,97.320886
28234,1.0,2.135756,18.079309,Monday,0.193932,-0.518779,-0.845606,-0.008%,-1.164756,0.179716,0.893962,-0.232957,5.375917,-0.965631,6.743603,84.016060,$546.057351945587,9.499990,29.197181,66.195584,-1.337996,3.759238,-1.428860,842.568246,14.239377,-0.523674,no,-0.087383,67.804301,1.042998,64.475670,29.496438,19.878034,0.096008,-1.651182,49.156413,61.876129,3368.956320,1.369307,70.801474,-1.117179,300.629990,1,September,1.638802,-0.409584,esurance,-0.548986,14.402071,9.467430,-0.650169,0.285636,-0.000234,45.379038,1.857392,-1.429478,-8.392614,2.138771,2.879075,5.850562,4.212254,no,2.923794,7.957879,1,107.559136
10906,0.0,2.314360,20.653557,Monday,-1.463253,-0.170908,-0.940822,0.0037%,1.911146,2.40072

In [15]:
dfCombine.isnull().sum()

y       0
x1      0
x2      0
x3      0
x4      0
x5      0
x6      0
x7      0
x8      0
x9      0
x10     0
x12     0
x13     0
x15     0
x17     0
x18     0
x19     0
x20     0
x21     0
x22     0
x23     0
x25     0
x26     0
x27     0
x28     0
x29     0
x31     0
x32     0
x34     0
x35     0
x36     0
x37     0
x40     0
x43     0
x46     0
x47     0
x48     0
x50     0
x51     0
x53     0
x56     0
x58     0
x59     0
x60     0
x62     0
x63     0
x65     0
x66     0
x67     0
x68     0
x69     0
x70     0
x71     0
x72     0
x73     0
x81     0
x82     0
x84     0
x87     0
x90     0
x92     0
x93     0
x94     0
x97     0
x98     0
x100    0
dtype: int64

### x3
Clean up day names

In [16]:
dfCombine["x3"].unique()

array(['Wed', 'Friday', 'Thursday', 'Tuesday', 'Sunday', 'Saturday',
       'Sat', 'Wednesday', 'Sun', 'Tue', 'Thur', 'Monday', 'Fri', 'Mon'],
      dtype=object)

In [17]:
dfCombine["x3"] = dfCombine["x3"].replace(["Mon"],"Monday")
dfCombine["x3"] = dfCombine["x3"].replace(["Tue"],"Tuesday")
dfCombine["x3"] = dfCombine["x3"].replace(["Wed"],"Wednesday")
dfCombine["x3"] = dfCombine["x3"].replace(["Thur"],"Thursday")
dfCombine["x3"] = dfCombine["x3"].replace(["Fri"],"Friday")
dfCombine["x3"] = dfCombine["x3"].replace(["Sat"],"Saturday")
dfCombine["x3"] = dfCombine["x3"].replace(["Sun"],"Sunday")

In [18]:
dfCombine["x3"].unique()

array(['Wednesday', 'Friday', 'Thursday', 'Tuesday', 'Sunday', 'Saturday',
       'Monday'], dtype=object)

### x7
Remove percent sign

In [19]:
dfCombine['x7'] = dfCombine["x7"].str.replace("%","")

In [20]:
dfCombine.head(5)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,$-908.650758424405,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,$-1864.9622875143,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,$967.007090837503,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935


In [21]:
# remove "E" notation
dfCombine["x7"] = pd.to_numeric(dfCombine["x7"])
dfCombine.sample(25)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
772,0.0,0.107077,20.552642,Tuesday,-1.437498,0.762980,-1.011265,-0.0041,1.070565,0.928667,2.869993,5.389915,2.871712,0.358573,4.313960,70.673609,$648.035204859616,4.697734,16.662983,16.663824,0.532781,3.928717,0.503306,871.175942,16.332748,2.136250,no,0.650047,48.297039,6.911581,66.885771,34.265413,18.705706,0.643017,2.673415,54.043287,54.219913,3002.441389,-0.486620,67.934342,1.769282,300.629990,0,August,3.914505,1.600965,esurance,1.557156,14.402071,14.367512,2.171735,0.770565,-0.000234,48.151438,3.236158,0.784291,-6.333024,2.138771,1.076207,2.590025,5.498653,no,6.718925,9.668421,1,100.884985
34307,0.0,1.273023,22.053456,Saturday,-1.994792,1.087270,-0.567472,-0.0217,-0.786781,1.053595,1.541803,4.117793,3.773072,1.463786,4.466748,77.506295,$-279.546902828793,3.733699,24.809930,79.702997,-0.732619,4.765305,1.506187,1121.416123,18.148320,0.424872,no,0.143344,51.047885,-0.288692,70.760656,26.564379,12.220737,-0.049707,-1.200092,49.646483,58.895985,3097.097972,-0.253352,67.451149,3.113694,300.629990,0,July,2.694016,-0.746819,progressive,1.898938,14.402071,17.321834,1.071393,0.904599,-0.000234,46.739731,0.030994,-0.676697,-7.024371,2.138771,-0.189121,2.241261,3.556115,no,3.199563,11.028675,0,101.542438
45443,0.0,4.608894,20.290190,Tuesday,3.078879,0.150438,-1.433616,-0.0167,-1.871372,1.518364,0.424728,3.317760,1.860178,-1.093891,5.942617,76.761835,$121.506772878465,11.770248,2.789985,29.893717,0.409159,2.885224,-1.397991,844.390705,24.377987,2.201065,no,1.846994,52.233326,0.574356,63.529444,32.603239,17.923862,0.551827,-1.054756,40.242319,60.952334,2615.092805,4.332628,75.745188,-0.393721,300.629990,0,October,5.559319,-0.237433,allstate,-0.952194,14.402071,15.852397,0.232255,0.269448,-0.000234,63.179629,18.012416,-0.227221,-5.625229,2.138771,0.376649,3.838252,7.967665,no,4.924121,9.213129,1,94.695188
44084,0.0,3.293018,19.157420,Saturday,1.420474,-0.219849,1.033762,0.0029,1.672068,1.399318,0.592411,0.526290,6.739052,-2.400671,9.153439,89.582923,$1192.93676103527,4.998687,14.236593,31.578573,-1.161219,6.862424,1.699156,1023.916700,11.447610,2.764890,no,0.761470,54.502193,-1.449897,74.255936,32.313597,10.523278,-0.414604,-1.956178,52.708958,60.813891,1197.585157,3.986109,74.474883,0.565610,300.629990,1,August,1.848658,0.454068,farmers,-1.778192,14.402071,11.310573,0.369809,0.492570,-0.000234,53.111738,8.272893,-0.854351,-3.703319,2.138771,0.363031,3.555995,3.635932,no,0.299973,9.287940,1,96.028951
28490,0.0,3.217393,17.342641,Monday,-1.478107,-1.953954,1.796983,0.0142,-0.439407,5.409125,1.306892,-0.666420,4.782358,3.103740,11.239363,77.580921,$-416.43469998121,3.407026,30.415046,114.368098,-1.099363,7.910148,0.410754,1044.215652,13.613744,2.860570,no,0.464802,59.144635,0.044094,72.771892,29.324017,14.023589,0.482145,-2.810914,46.649672,65.975224,3912.105127,3.650234,60.533486,-0.408702,300.629990,0,August,3.779113,-1.129684,geico,-1.162089,14.402071,22.358326,-1.703463,1.031835,2.184266,42.964469,7.677909,-0.683260,-8.817523,1.339565,-0.339142,4.937237,5.504181,no,2.385184,12.520784,1,102.484860
13044,1.0,2.235121,21.351481,Friday,-0.408905,0.034197,1.207943,-0.0043,0.006180,3.896382,0.818554,1.317243,1.281014,-0.115567,9.387014,85.528016,$-1253.44661043393,3.022038,16.549574,70.529865,-0.529502,7.681131,-1.509386,923.614042,15.639400,-0.096924,no,-0.311253,54.492445,2.115765,72.741412,30.943619,17.693616,1.083433,-3.537823,47.360301,68.887962,2825.339262,4.306704,64.916397,0.227628,300.629990,0,August,4.919343,1.171348,allstate,-2.292720,7.751741,11.856100,0.792739,0.377526,-0.000234,46.996587,16.243861,0.115853,-5.963057,2.138771,0.186529,2.177430,1.401721,no,4.434108,9.624344,0,105.713276
11566,0.0,3.566892,18.354018,Tuesday,0.370415,-1.352613,1.786478,0.0107,0.497107,2.393512,-0.

### x19

In [22]:
# replace dollar sign
dfCombine["x19"] = dfCombine["x19"].replace(to_replace="[\$]",value="",regex=True)
dfCombine.head(10)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758424405,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.9622875143,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187402955527,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626380634258,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007090837503,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163475698959,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,no,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,no,4.183036,5.632881,0,97.129172
6,0.0,0.094162,18.894755,Thursday,2.027918,1.580408,1.190932,0.0174,-0.331730,2.178622,-0.454172,1.762009,2.125462,-0.907832,8.6

### Round to 6 decimal places

In [23]:
dfCombine["x19"] = dfCombine["x19"].astype(float).round(6)
dfCombine.head(10)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,no,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,no,4.183036,5.632881,0,97.129172
6,0.0,0.094162,18.894755,Thursday,2.027918,1.580408,1.190932,0.0174,-0.331730,2.178622,-0.454172,1.762009,2.125462,-0.907832,8.631917,71.282134,749.965010,14.2208

### x31

In [24]:
# map 0 to no and 1 to yes
dfCombine["x31"] = dfCombine["x31"].map({"no":0,"yes":1})
dfCombine["x31"].unique()

array([0, 1], dtype=int64)

### x93

In [25]:
dfCombine["x93"] = dfCombine["x93"].map({"no":0,"yes":1})
dfCombine["x93"].unique()

array([0, 1], dtype=int64)

### Encode categorical variables

### x3

In [26]:
# days of the week
x3 = pd.get_dummies(dfCombine['x3'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x3,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,0,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,0,4.183036,5.632881,0,97.129172,0,0,1,0,0,0,0


#### x60

In [27]:
# months
x60 = pd.get_dummies(dfCombine['x60'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x60,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday,cat__April,cat__August,cat__December,cat__February,cat__January,cat__July,cat__June,cat__March,cat__May,cat__November,cat__October,cat__September
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,0,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.

#### x65

In [28]:
# insurance name
x65 = pd.get_dummies(dfCombine['x65'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x65,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday,cat__April,cat__August,cat__December,cat__February,cat__January,cat__July,cat__June,cat__March,cat__May,cat__November,cat__October,cat__September,cat__allstate,cat__esurance,cat__farmers,cat__geico,cat__progressive
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.1

In [29]:
dfTrainClean = dfCombine.iloc[:40000,]
dfTestClean = dfCombine.iloc[40000:,]

### Logistic regression (classification) & neural network

class sklearn.linear_model.LogisticRegression(penalty='l2',dual=False,tol=0.0001,C=1.0,fit_intercept=True,
intercept_scaling=1,class_weight=None,random_state=None,solver='lbfgs',max_iter=100,multi_class='auto', 
verbose=0,warm_start=False,n_jobs=None,l1_ratio=None)

class sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(100, ), activation='relu', *, 
solver='adam',alpha=0.0001,batch_size='auto', learning_rate='constant', learning_rate_init=0.001,
power_t=0.5,max_iter=200,shuffle=True, random_state=None, tol=0.0001, verbose=False, 
warm_start=False, momentum=0.9,nesterovs_momentum=True, early_stopping=False, 
validation_fraction=0.1,beta_1=0.9,beta_2=0.999,epsilon=1e-08,n_iter_no_change=10, max_fun=15000)

In [31]:
dfCombine.columns

Index(['y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x12',
       'x13', 'x15', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x25',
       'x26', 'x27', 'x28', 'x29', 'x31', 'x32', 'x34', 'x35', 'x36', 'x37',
       'x40', 'x43', 'x46', 'x47', 'x48', 'x50', 'x51', 'x53', 'x56', 'x58',
       'x59', 'x60', 'x62', 'x63', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70',
       'x71', 'x72', 'x73', 'x81', 'x82', 'x84', 'x87', 'x90', 'x92', 'x93',
       'x94', 'x97', 'x98', 'x100', 'cat__Friday', 'cat__Monday',
       'cat__Saturday', 'cat__Sunday', 'cat__Thursday', 'cat__Tuesday',
       'cat__Wednesday', 'cat__April', 'cat__August', 'cat__December',
       'cat__February', 'cat__January', 'cat__July', 'cat__June', 'cat__March',
       'cat__May', 'cat__November', 'cat__October', 'cat__September',
       'cat__allstate', 'cat__esurance', 'cat__farmers', 'cat__geico',
       'cat__progressive'],
      dtype='object')

In [32]:
features = [
    'x1','x2','x4','x5','x6','x7','x8','x9','x10','x12',
    'x13','x15','x17','x18', 'x19', 'x20', 'x21','x22', 'x23', 'x25',
    'x26','x27','x28','x29', 'x31', 'x32', 'x34','x35', 'x36', 'x37',
    'x40','x43','x46','x47', 'x48', 'x50', 'x51','x53', 'x56', 'x58',
    'x59','x62','x63','x66', 'x67', 'x68', 'x69','x70',
    'x71','x72','x73','x81', 'x82', 'x84', 'x87','x90','x92','x93',
    'x94','x97','x98', 'x100', 'cat__Friday', 'cat__Monday',
    'cat__Saturday','cat__Sunday','cat__Thursday', 'cat__Tuesday',
    'cat__Wednesday','cat__April','cat__August','cat__December',
    'cat__February','cat__January','cat__July','cat__June', 'cat__March',
    'cat__May','cat__November', 'cat__October', 'cat__September',
    'cat__allstate','cat__esurance', 'cat__farmers', 'cat__geico',
    'cat__progressive']

# x4 x6 x9 x10 x12 x17 x22 x23 x34 x48 x53 x59 x69 x73 x81 x92 x97
dropFeatures = [
    'x1','x2','x5','x7','x8',
    'x13','x15','x18','x19', 'x20','x21','x25',
    'x26','x27','x28','x29', 'x31','x32','x35', 'x36', 'x37',
    'x40','x43','x46','x47', 'x50','x51', 'x56','x58',
    'x62','x63','x66', 'x67', 'x68','x70',
    'x71','x72','x82','x84', 'x87','x90','x93',
    'x94','x98', 'x100', 'cat__Friday', 'cat__Monday',
    'cat__Saturday','cat__Sunday','cat__Thursday', 'cat__Tuesday',
    'cat__Wednesday','cat__April','cat__August','cat__December',
    'cat__February','cat__January','cat__July','cat__June', 'cat__March',
    'cat__May','cat__November', 'cat__October', 'cat__September',
    'cat__allstate','cat__esurance', 'cat__farmers', 'cat__geico',
    'cat__progressive']

### Scale data

In [35]:
scalerX = StandardScaler()
scalerX.fit(dfTrainClean[dropFeatures])
xTrainScaled = scalerX.transform(dfTrainClean[dropFeatures])
xTestScaled = scalerX.transform(dfTestClean[dropFeatures])

In [36]:
# logistic regression
logReg = LogisticRegression(C=7.7777,solver="lbfgs",max_iter=1111,tol=0.000003333, \
    multi_class="ovr",class_weight=None)

logReg.fit(xTrainScaled.astype(float),dfTrainClean["y"].astype(int))

LogisticRegression(C=7.7777, max_iter=1111, multi_class='ovr', tol=3.333e-06)

In [37]:
# neural networks
mlp = MLPClassifier(hidden_layer_sizes=(76,76,),random_state=None,alpha=0.1111, \
    solver='sgd',learning_rate="adaptive",activation="relu",n_iter_no_change=10, \
    tol=0.00001111,learning_rate_init=0.001111,max_iter=253,momentum=0.9111, \
    batch_size="auto",shuffle=True,warm_start=False,early_stopping=False,validation_fraction=0.1)

mlp.fit(xTrainScaled.astype(float),dfTrainClean["y"].astype(int))

C:\Users\TamLe\anaconda3\envs\sports\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (253) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.1111, hidden_layer_sizes=(76, 76),
              learning_rate='adaptive', learning_rate_init=0.001111,
              max_iter=253, momentum=0.9111, solver='sgd', tol=1.111e-05)

### Predictions

In [38]:
yPredTrainLog = logReg.predict(xTrainScaled.astype(float))
yPredTestLog = logReg.predict(xTestScaled.astype(float))
yProbTrainLog = logReg.predict_proba(xTrainScaled.astype(float))
yProbTestLog = logReg.predict_proba(xTestScaled.astype(float))

In [39]:
yPredTrainMLP = mlp.predict(xTrainScaled.astype(float))
yPredTestMLP = mlp.predict(xTestScaled.astype(float))
yProbTrainMLP = mlp.predict_proba(xTrainScaled.astype(float))
yProbTestMLP = mlp.predict_proba(xTestScaled.astype(float))

In [40]:
pd.DataFrame(yPredTrainLog).value_counts()

0    39238
1      762
dtype: int64

In [41]:
pd.DataFrame(dfTrain["y"]).value_counts()

y
0    34197
1     5803
dtype: int64

### Probabilities on training set

In [42]:
pd.DataFrame(yProbTrainLog).head(20)

,0,1
0,0.959971,0.040029
1,0.658801,0.341199
2,0.828818,0.171182
3,0.549976,0.450024
4,0.983968,0.016032
5,0.888512,0.111488
6,0.958273,0.041727
7,0.859887,0.140113
8,0.924804,0.075196
9,0.854813,0.145187


In [43]:
pd.DataFrame(yProbTestLog).tail(25)

,0,1
9975,0.946557,0.053443
9976,0.909108,0.090892
9977,0.808916,0.191084
9978,0.656960,0.343040
9979,0.330718,0.669282
9980,0.838664,0.161336
9981,0.948758,0.051242
9982,0.948408,0.051592
9983,0.943232,0.056768
9984,0.768390,0.231610


In [44]:
pd.DataFrame(yProbTestLog).iloc[:,[1]].head(15)

,1
0,0.020053
1,0.027552
2,0.259752
3,0.177349
4,0.013140
5,0.057507
6,0.274311
7,0.059734
8,0.326363
9,0.623363


### Save results

In [45]:
pd.DataFrame(yProbTestLog).iloc[:,[1]].to_csv("glmresults.csv",header=False,index=False)
pd.DataFrame(yProbTestMLP).iloc[:,[1]].to_csv("nonglmresults.csv",header=False,index=False)

In [46]:
print(len(yProbTestMLP))

10000


### Statsmodels

In [49]:
xInclConstant = sm.add_constant(dfTrainClean[dropFeatures])
model = sm.Logit(dfTrainClean["y"].astype(int),xInclConstant.astype(float))
# logistic regression
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.359918
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                40000
Model:                          Logit   Df Residuals:                    39933
Method:                           MLE   Df Model:                           66
Date:                Fri, 04 Jun 2021   Pseudo R-squ.:                  0.1308
Time:                        18:55:35   Log-Likelihood:                -14397.
converged:                       True   LL-Null:                       -16563.
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                3.5752   2.06e+06   1.73e-06      1.000   -4.04e+06    4.04e+06
x1                   0.0259      0.008      3.294      0.001       0.011       0.041
x2                   0.0954      0.022      4.407      0.000       0.053       0.138
x5                   0.0446      0.016      2.826      0.005       0.014       0.076
x7                 -74.1062      1.636    -45.290      0.000     -77.313     -70.899
x8                   0.1096      0.018      6.179      0.000       0.075       0.144
x13                  0.0194      0.009      2.108      0.035       0.001       0.037
x15                 -0.0205      0.014     -1.494      0.135      -0.047       0.006
x18                 -0.0094      0.005     -2.034      0.042      -0.018      -0.000
x19                 -0.0001   1.51e-05     -9.304      0.000      -0.000      -0.000
x20                 -0.0301      0.006     -4.903      0.000      -0.042      -0.018
x21                  0.0012      0.001      1.199      0.231      -0.001       0.003
x25                  0.0058      0.006      0.910      0.363      -0.007       0.018
x26                  0.0143      0.014      1.050      0.294      -0.012       0.041
x27                 -0.0004      0.000     -1.429      0.153      -0.001       0.000
x28                 -0.0973      0.012     -7.867      0.000      -0.122      -0.073
x29                 -0.0202      0.013     -1.574      0.115      -0.045       0.005
x31                 -0.5720      0.059     -9.740      0.000      -0.687      -0.457
x32                  0.0144      0.009      1.642      0.101      -0.003       0.032
x35                  0.0243      0.015      1.670      0.095      -0.004       0.053
x36                 -0.0115      0.010     -1.209      0.227      -0.030       0.007
x37                  0.0623      0.009      6.603      0.000       0.044       0.081
x40                 -0.1068      0.008    -13.138      0.000      -0.123      -0.091
x43                 -0.0168      0.015     -1.118      0.264      -0.046       0.013
x46                 -0.0549      0.012     -4.517      0.000      -0.079      -0.031
x47                 -0.1394      0.008    -16.839      0.000      -0.156      -0.123
x50              -9.406e-05   2.48e-05     -3.796      0.000      -0.000   -4.55e-05
x51                  0.0436      0.010      4.290      0.000       0.024       0.063
x56                 -0.0230      0.011     -2.158      0.031      -0.044      -0.002
x58                  0.0007      0.000      1.596      0.110      -0.000       0.002
x62                 -0.0533      0.013     -3.975      0.000      -0.080      -0.027
x63                  0.0444      0.013      3.485      0.000       0.019       0.069
x66                  0.0689      0.016      4.193      0.000       0.037       0.101
x67                 -0.0254      0.014     -1.867      0.062      -0.052       0.001
x68                  0.0145      0.003      4.265      0.000       0.008       0.021
x70        

### AUC curve on training set
There is no dependent variable in test set.

### Logistic regresion score

In [53]:
yTrainClean = dfTrainClean["y"].to_numpy()
print(yTrainClean)
falsePositiveRate,truePositiveRate,threshold = metrics.roc_curve(yTrainClean,yProbTrainLog[:,1])

rocAucScore = metrics.roc_auc_score(yTrainClean,yProbTrainLog[:,1])
print("auc score: %s" % rocAucScore)
logLoss = log_loss(yTrainClean,yProbTrainLog[:,1])
print("log loss: %s" % logLoss)

[0. 1. 1. ... 0. 0. 0.]
auc score: 0.7527958790394674
log loss: 0.35991762698348395


### Neural networks score

In [54]:
yTrainClean = dfTrainClean["y"].to_numpy()
print(yTrainClean)
falsePositiveRate,truePositiveRate,threshold = metrics.roc_curve(yTrainClean,yProbTrainMLP[:,1])

rocAucScore = metrics.roc_auc_score(yTrainClean,yProbTrainMLP[:,1])
print("auc score: %s" % rocAucScore)
logLoss = log_loss(yTrainClean,yProbTrainMLP[:,1])
print("log loss: %s" % logLoss)

[0. 1. 1. ... 0. 0. 0.]
auc score: 0.8323815264437423
log loss: 0.3154787944414712
